In [113]:
import keras


In [114]:
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK,Trials,fmin,tpe,hp

In [115]:
from sklearn.metrics import mean_squared_error

In [116]:
from sklearn.model_selection import train_test_split

In [117]:
import mlflow
from mlflow.models import infer_signature

In [118]:
data=pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [119]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [120]:

train,test=train_test_split(data,test_size=0.25,random_state=42)
train

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
2835,6.3,0.25,0.22,3.30,0.048,41.0,161.0,0.99256,3.16,0.50,10.5,6
1157,7.8,0.30,0.29,16.85,0.054,23.0,135.0,0.99980,3.16,0.38,9.0,6
744,7.4,0.38,0.27,7.50,0.041,24.0,160.0,0.99535,3.17,0.43,10.0,5
1448,7.4,0.16,0.49,1.20,0.055,18.0,150.0,0.99170,3.23,0.47,11.2,6
3338,7.2,0.27,0.28,15.20,0.046,6.0,41.0,0.99665,3.17,0.39,10.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4426,6.2,0.21,0.52,6.50,0.047,28.0,123.0,0.99418,3.22,0.49,9.9,6
466,7.0,0.14,0.32,9.00,0.039,54.0,141.0,0.99560,3.22,0.43,9.4,6
3092,7.6,0.27,0.52,3.20,0.043,28.0,152.0,0.99129,3.02,0.53,11.4,6
3772,6.3,0.24,0.29,13.70,0.035,53.0,134.0,0.99567,3.17,0.38,10.6,6


In [121]:
train[['quality']].values.ravel()

array([6, 6, 5, ..., 6, 6, 8], dtype=int64)

In [122]:
train_x=train.drop(['quality'],axis=1).values
train_y=train[['quality']].values.ravel()

## test dataset
X_test=test.drop(['quality'],axis=1).values
y_test=test[['quality']].values.ravel()

## splitting this train data into train and validation

X_train,X_val,y_train,y_val=train_test_split(train_x,train_y,test_size=0.20,random_state=42)

signature=infer_signature(train_x,train_y)

In [123]:
# X_train,X_test,y_train,y_test = train_test_split(data.drop("quality",axis=1),data['quality'],test_size=0.2,random_state=42)

In [124]:
# X_train,X_val,y_train,y_val = train_test_split(X_train,y_train,test_size=0.2,random_state=42)

In [125]:
signature = infer_signature(X_train,y_train)
print("X_train shape:", X_train.shape)  # Should be (n_samples, n_features)
print("y_train shape:", y_train.shape)  

X_train shape: (2938, 11)
y_train shape: (2938,)


In [130]:
print("X_train shape:",type( X_train))  # Should be (n_samples, n_features)
print("y_train shape:", type(y_train)) 

X_train shape: <class 'numpy.ndarray'>
y_train shape: <class 'numpy.ndarray'>


In [126]:
def train_model(params,epochs,X_train,y_train,X_val,y_val,X_test,y_test):
    
    mean = np.mean(X_train,axis = 0)
    var = np.var(X_train,axis = 0)
    
    model = keras.Sequential(
        [
            keras.Input([X_train.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64,activation="relu"),
            keras.layers.Dense(1)
        ]
    )
    
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params["lr"],
        momentum=params['momentum'],
    
    ),
    loss = "mean_squared_error",
    metrics=[keras.metrics.RootMeanSquaredError()]
    )
    
    with mlflow.start_run(nested=True):
        model.fit(X_train,y_train,validation_data=(X_val,y_val),
                  epochs=epochs,
                  batch_size=64)
        
        eval = model.evaluate(X_val,y_val,batch_size=64)
        
        eval_rmse = eval[1]
        
        mlflow.log_params(params)
        mlflow.log_metric("rmse",eval_rmse)
        mlflow.tensorflow.log_model(model,"model",signature=signature)
        
        return {"loss":eval_rmse,"status":STATUS_OK,"model":model}
    

In [127]:
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        X_train=X_train,
        y_train=y_train,
        X_val=X_val,
        y_val=y_val,
        X_test=X_test,
        y_test=y_test,
    )
    return result

In [128]:
space={
    "lr":hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),
    "momentum":hp.uniform("momentum",0.0,1.0)
}

In [129]:
mlflow.set_experiment("/wine-qual")
with mlflow.start_run():
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )
    
    best_run = sorted(trials.results,key=lambda x : x['loss'])[0]
    
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse",best_run['loss'])
    mlflow.tensorflow.log_model(best_run['model'],"model",signature=signature)
    
    print("best params",best)
    print("best eval rmse",best_run['loss'])
    

Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 15s 334ms/step - loss: 35.9898 - root_mean_squared_error: 5.9991
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 34.2357 - root_mean_squared_error: 5.8511 - val_loss: 33.7971 - val_root_mean_squared_error: 5.8135

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 32.4698 - root_mean_squared_error: 5.6982
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 32.9855 - root_mean_squared_error: 5.7433 - val_loss: 32.7494 - val_root_mean_squared_error: 5.7227

Epoch 3/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 32.6104 - root_mean_squared_error: 5.7106
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 32.0917 - root_mean_squared_error: 5.6649 - val_loss: 31.7352 - val_root_mean_squared_error: 5.6334

 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 31.3061 - root_mean_squared_error: 5.5952
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s

In [131]:
from mlflow.models import validate_serving_input

model_uri = 'runs:/a4551550d7024e06a46f6ddf248883db/model'

# The logged model does not contain an input_example.
# Manually generate a serving payload to verify your model prior to deployment.
from mlflow.models import convert_input_example_to_serving_input

# Define INPUT_EXAMPLE via assignment with your own input example to the model
# A valid input example is a data instance suitable for pyfunc prediction
serving_payload = convert_input_example_to_serving_input(X_test)

# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)

C:\Users\kevin\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


array([[6.284259 ],
       [7.0843887],
       [6.4947004],
       ...,
       [6.229784 ],
       [6.94911  ],
       [5.87115  ]], dtype=float32)

In [133]:
import mlflow
logged_model = 'runs:/a4551550d7024e06a46f6ddf248883db/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(pd.DataFrame(X_test))

39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


array([[6.284259 ],
       [7.0843887],
       [6.4947004],
       ...,
       [6.229784 ],
       [6.94911  ],
       [5.87115  ]], dtype=float32)